Nama: Firdatul A'yuni

NIM: 210411100144

Kelas: PPW B

# **Vector Space Model**

## Pengertian Vector Space Model

Vector Space Model (VSM) adalah model matematika yang digunakan dalam pengolahan informasi dan pencarian data, seperti di mesin pencari, untuk merepresentasikan teks (dokumen atau kata-kata) dalam bentuk vektor di ruang berdimensi banyak. Ini adalah cara untuk menggambarkan dokumen atau teks sebagai titik dalam ruang vektor, yang memungkinkan kita mengukur seberapa mirip dua dokumen atau teks.

## Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd

df_awal = pd.read_csv("/content/drive/MyDrive/KULIAH/SEMESTER 7/PPW/data-berita-2kategori.csv")
df_awal.head()

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,Kedubes RI di Tunisia Ajak Jelajahi Keindahan ...,"TIMESINDONESIA, BANYUWANGI – Kedutaan Besar Re...",08/09/2024 - 19:47,Wisata
1,Rekomendasi Hotel Bintang 5 di Kawasan Puncak ...,"TIMESINDONESIA, CIANJUR – Berlibur atau melaku...",08/09/2024 - 09:00,Wisata
2,Menikmati Sensasi Petik Melon di Desa Balongwo...,"TIMESINDONESIA, MOJOKERTO – Ketika bosan denga...",07/09/2024 - 13:09,Wisata
3,Sejumlah Influencer Kepincut Pelayanan dan Fas...,"TIMESINDONESIA, BANYUWANGI – Luminor Hotel Ban...",07/09/2024 - 06:08,Wisata
4,5 Pantai Unik dan Cantik Masuk Kawasan TN Mata...,"TIMESINDONESIA, SUMBA TIMUR – Ternyata di Pula...",01/09/2024 - 14:16,Wisata


## Preprocessing

Praproses pada data teks adalah langkah-langkah awal yang dilakukan untuk membersihkan dan menyiapkan teks sebelum dianalisis lebih lanjut. Tujuan dari praproses adalah membuat data teks lebih mudah diolah dan lebih berguna untuk model analisis atau pemrosesan. Praproses teks biasanya terdiri dari Cleansing, Casefolding, Tokenization, Stopword Removal, Stemming. Tujuan dilakukan preprocessing adalah untuk meningkatkan akurasi, mengurangi kebisingan dan mempermudah analisis.

## Install dan Import Library yang diperlukan

In [7]:
pip install pandas scikit-learn nltk unidecode

In [9]:
 !pip install PySastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 1.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from unidecode import unidecode
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

**CLEANSING**

Cleansing adalah proses pembersihan data teks dari elemen-elemen yang tidak diinginkan atau tidak relevan. Tujuannya adalah untuk mengurangi "kebisingan" dalam data dan membuat teks lebih bersih sehingga analisis selanjutnya dapat dilakukan dengan lebih akurat. Cleansing yang dilakukan disini meliputi:
1. Menghapus URL
2. Menghapus tag HTML
3. Menghapus emoji
4. Menghapus angka
5. Menghapus simbol


In [3]:
import string
#menghapus url
def remove_url(news):
  url = re.compile(r'https?://\S+|www\.S+')
  return url.sub(r'', news)
#menghapus html
def remove_html(news):
  html = re.compile(r'<.#?>')
  return html.sub(r'', news)
#menghapus emoji
def remove_emoji(news):
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF""]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', news)
#menghapus angka
def remove_numbers(news):
  news = re.sub(r'\d+', '', news)
  return news
#menghapus simbol
def remove_symbols(news):
  news = re.sub(r'[^a-zA-Z0-9\s]', '', news)
  return news

df_awal['cleansing'] = df_awal['Isi Berita'].apply(lambda x: remove_url(x))
df_awal['cleansing'] = df_awal['cleansing'].apply(lambda x: remove_html(x))
df_awal['cleansing'] = df_awal['cleansing'].apply(lambda x: remove_emoji(x))
df_awal['cleansing'] = df_awal['cleansing'].apply(lambda x: remove_symbols(x))
df_awal['cleansing'] = df_awal['cleansing'].apply(lambda x: remove_numbers(x))

df_awal.head(5)

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,cleansing
0,Kedubes RI di Tunisia Ajak Jelajahi Keindahan ...,"TIMESINDONESIA, BANYUWANGI – Kedutaan Besar Re...",08/09/2024 - 19:47,Wisata,TIMESINDONESIA BANYUWANGI Kedutaan Besar Repu...
1,Rekomendasi Hotel Bintang 5 di Kawasan Puncak ...,"TIMESINDONESIA, CIANJUR – Berlibur atau melaku...",08/09/2024 - 09:00,Wisata,TIMESINDONESIA CIANJUR Berlibur atau melakuka...
2,Menikmati Sensasi Petik Melon di Desa Balongwo...,"TIMESINDONESIA, MOJOKERTO – Ketika bosan denga...",07/09/2024 - 13:09,Wisata,TIMESINDONESIA MOJOKERTO Ketika bosan dengan ...
3,Sejumlah Influencer Kepincut Pelayanan dan Fas...,"TIMESINDONESIA, BANYUWANGI – Luminor Hotel Ban...",07/09/2024 - 06:08,Wisata,TIMESINDONESIA BANYUWANGI Luminor Hotel Banyu...
4,5 Pantai Unik dan Cantik Masuk Kawasan TN Mata...,"TIMESINDONESIA, SUMBA TIMUR – Ternyata di Pula...",01/09/2024 - 14:16,Wisata,TIMESINDONESIA SUMBA TIMUR Ternyata di Pulau ...


**CASEFOLDING**

Casefolding adalah proses mengubah semua huruf dalam teks menjadi huruf kecil, Contohnya "Makan" menjadi "makan". Tujuannya adalah untuk memastikan bahwa analisis teks tidak dipengaruhi oleh perbedaan antara huruf kapital dan huruf kecil. Dibawah ini adalah fungsi untuk proses casefolding

In [4]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df_awal ['case_folding'] = df_awal['cleansing'].apply(case_folding)

df_awal.head(5)

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,cleansing,case_folding
0,Kedubes RI di Tunisia Ajak Jelajahi Keindahan ...,"TIMESINDONESIA, BANYUWANGI – Kedutaan Besar Re...",08/09/2024 - 19:47,Wisata,TIMESINDONESIA BANYUWANGI Kedutaan Besar Repu...,timesindonesia banyuwangi kedutaan besar repu...
1,Rekomendasi Hotel Bintang 5 di Kawasan Puncak ...,"TIMESINDONESIA, CIANJUR – Berlibur atau melaku...",08/09/2024 - 09:00,Wisata,TIMESINDONESIA CIANJUR Berlibur atau melakuka...,timesindonesia cianjur berlibur atau melakuka...
2,Menikmati Sensasi Petik Melon di Desa Balongwo...,"TIMESINDONESIA, MOJOKERTO – Ketika bosan denga...",07/09/2024 - 13:09,Wisata,TIMESINDONESIA MOJOKERTO Ketika bosan dengan ...,timesindonesia mojokerto ketika bosan dengan ...
3,Sejumlah Influencer Kepincut Pelayanan dan Fas...,"TIMESINDONESIA, BANYUWANGI – Luminor Hotel Ban...",07/09/2024 - 06:08,Wisata,TIMESINDONESIA BANYUWANGI Luminor Hotel Banyu...,timesindonesia banyuwangi luminor hotel banyu...
4,5 Pantai Unik dan Cantik Masuk Kawasan TN Mata...,"TIMESINDONESIA, SUMBA TIMUR – Ternyata di Pula...",01/09/2024 - 14:16,Wisata,TIMESINDONESIA SUMBA TIMUR Ternyata di Pulau ...,timesindonesia sumba timur ternyata di pulau ...


**TOKENIZATION**

Tokenisasi adalah proses memecah teks menjadi bagian-bagian kecil yang disebut token, contohnya kalimat "adi makan nasi" menjadi ["adi", "makan", "nasi"].

Token bisa berupa kata, frasa, atau simbol tergantung pada tujuan analisis. Tujuan dari tokenisasi adalah untuk mempermudah analisis teks dengan mengubah teks panjang menjadi unit-unit yang lebih kecil dan lebih mudah dikelola.

In [5]:
def tokenize(text):
    tokens = text.split()
    return tokens

df_awal['tokenize'] = df_awal['case_folding'].apply(tokenize)

df_awal.head(5)

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,cleansing,case_folding,tokenize
0,Kedubes RI di Tunisia Ajak Jelajahi Keindahan ...,"TIMESINDONESIA, BANYUWANGI – Kedutaan Besar Re...",08/09/2024 - 19:47,Wisata,TIMESINDONESIA BANYUWANGI Kedutaan Besar Repu...,timesindonesia banyuwangi kedutaan besar repu...,"[timesindonesia, banyuwangi, kedutaan, besar, ..."
1,Rekomendasi Hotel Bintang 5 di Kawasan Puncak ...,"TIMESINDONESIA, CIANJUR – Berlibur atau melaku...",08/09/2024 - 09:00,Wisata,TIMESINDONESIA CIANJUR Berlibur atau melakuka...,timesindonesia cianjur berlibur atau melakuka...,"[timesindonesia, cianjur, berlibur, atau, mela..."
2,Menikmati Sensasi Petik Melon di Desa Balongwo...,"TIMESINDONESIA, MOJOKERTO – Ketika bosan denga...",07/09/2024 - 13:09,Wisata,TIMESINDONESIA MOJOKERTO Ketika bosan dengan ...,timesindonesia mojokerto ketika bosan dengan ...,"[timesindonesia, mojokerto, ketika, bosan, den..."
3,Sejumlah Influencer Kepincut Pelayanan dan Fas...,"TIMESINDONESIA, BANYUWANGI – Luminor Hotel Ban...",07/09/2024 - 06:08,Wisata,TIMESINDONESIA BANYUWANGI Luminor Hotel Banyu...,timesindonesia banyuwangi luminor hotel banyu...,"[timesindonesia, banyuwangi, luminor, hotel, b..."
4,5 Pantai Unik dan Cantik Masuk Kawasan TN Mata...,"TIMESINDONESIA, SUMBA TIMUR – Ternyata di Pula...",01/09/2024 - 14:16,Wisata,TIMESINDONESIA SUMBA TIMUR Ternyata di Pulau ...,timesindonesia sumba timur ternyata di pulau ...,"[timesindonesia, sumba, timur, ternyata, di, p..."


**STOPWORD REMOVAL**

Stopwords adalah kata-kata umum yang sering muncul dalam teks tetapi tidak memberikan banyak informasi untuk analisis. Contoh stopwords dalam bahasa Indonesia termasuk "dan", "atau", "yang". Pada fungsi dibawah ini adalah fungsi untuk melakukan stopword removal/menghapus stopword

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

df_awal['no_stopwords'] = df_awal['tokenize'].apply(remove_stopwords)
df_awal.head()

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,cleansing,case_folding,tokenize,no_stopwords
0,Kedubes RI di Tunisia Ajak Jelajahi Keindahan ...,"TIMESINDONESIA, BANYUWANGI – Kedutaan Besar Re...",08/09/2024 - 19:47,Wisata,TIMESINDONESIA BANYUWANGI Kedutaan Besar Repu...,timesindonesia banyuwangi kedutaan besar repu...,"[timesindonesia, banyuwangi, kedutaan, besar, ...","[timesindonesia, banyuwangi, kedutaan, republi..."
1,Rekomendasi Hotel Bintang 5 di Kawasan Puncak ...,"TIMESINDONESIA, CIANJUR – Berlibur atau melaku...",08/09/2024 - 09:00,Wisata,TIMESINDONESIA CIANJUR Berlibur atau melakuka...,timesindonesia cianjur berlibur atau melakuka...,"[timesindonesia, cianjur, berlibur, atau, mela...","[timesindonesia, cianjur, berlibur, perjalanan..."
2,Menikmati Sensasi Petik Melon di Desa Balongwo...,"TIMESINDONESIA, MOJOKERTO – Ketika bosan denga...",07/09/2024 - 13:09,Wisata,TIMESINDONESIA MOJOKERTO Ketika bosan dengan ...,timesindonesia mojokerto ketika bosan dengan ...,"[timesindonesia, mojokerto, ketika, bosan, den...","[timesindonesia, mojokerto, bosan, buah, melon..."
3,Sejumlah Influencer Kepincut Pelayanan dan Fas...,"TIMESINDONESIA, BANYUWANGI – Luminor Hotel Ban...",07/09/2024 - 06:08,Wisata,TIMESINDONESIA BANYUWANGI Luminor Hotel Banyu...,timesindonesia banyuwangi luminor hotel banyu...,"[timesindonesia, banyuwangi, luminor, hotel, b...","[timesindonesia, banyuwangi, luminor, hotel, b..."
4,5 Pantai Unik dan Cantik Masuk Kawasan TN Mata...,"TIMESINDONESIA, SUMBA TIMUR – Ternyata di Pula...",01/09/2024 - 14:16,Wisata,TIMESINDONESIA SUMBA TIMUR Ternyata di Pulau ...,timesindonesia sumba timur ternyata di pulau ...,"[timesindonesia, sumba, timur, ternyata, di, p...","[timesindonesia, sumba, timur, pulau, sumba, p..."


**STEMMING**

Stemming adalah proses mengubah kata ke bentuk dasarnya atau akar kata. Tujuan utama dari stemming adalah untuk menyederhanakan kata-kata yang memiliki variasi morfologis (bentuk yang berbeda tetapi makna yang sama) menjadi bentuk dasar yang konsisten. Contoh "berlari" menjadi "lari".

Stemming membantu dalam mengurangi kompleksitas data teks dengan mengelompokkan berbagai bentuk kata ke dalam satu bentuk dasar, sehingga memudahkan analisis.

In [12]:
# Inisialisasi Stemmer Bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(tokenize):
    """
    Fungsi untuk melakukan stemming pada list token.

    Args:
        tokens (list of str): List kata/token yang akan di-stem.

    Returns:
        list of str: List kata/token setelah di-stem.
    """
    return [stemmer.stem(word) for word in tokenize]

# Terapkan fungsi stemming pada data
df_awal['stemmed'] = df_awal['no_stopwords'].apply(stem_text)

# Gabungkan token yang sudah di-stem menjadi teks
df_awal['stemmed_text'] = df_awal['stemmed'].apply(lambda x: ' '.join(x))

# Tampilkan hasil
df_awal.head(5)


,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,cleansing,case_folding,tokenize,no_stopwords,stemmed,stemmed_text
0,Kedubes RI di Tunisia Ajak Jelajahi Keindahan ...,"TIMESINDONESIA, BANYUWANGI – Kedutaan Besar Re...",08/09/2024 - 19:47,Wisata,TIMESINDONESIA BANYUWANGI Kedutaan Besar Repu...,timesindonesia banyuwangi kedutaan besar repu...,"[timesindonesia, banyuwangi, kedutaan, besar, ...","[timesindonesia, banyuwangi, kedutaan, republi...","[timesindonesia, banyuwangi, duta, republik, i...",timesindonesia banyuwangi duta republik indone...
1,Rekomendasi Hotel Bintang 5 di Kawasan Puncak ...,"TIMESINDONESIA, CIANJUR – Berlibur atau melaku...",08/09/2024 - 09:00,Wisata,TIMESINDONESIA CIANJUR Berlibur atau melakuka...,timesindonesia cianjur berlibur atau melakuka...,"[timesindonesia, cianjur, berlibur, atau, mela...","[timesindonesia, cianjur, berlibur, perjalanan...","[timesindonesia, cianjur, libur, jalan, bisnis...",timesindonesia cianjur libur jalan bisnis cian...
2,Menikmati Sensasi Petik Melon di Desa Balongwo...,"TIMESINDONESIA, MOJOKERTO – Ketika bosan denga...",07/09/2024 - 13:09,Wisata,TIMESINDONESIA MOJOKERTO Ketika bosan dengan ...,timesindonesia mojokerto ketika bosan dengan ...,"[timesindonesia, mojokerto, ketika, bosan, den...","[timesindonesia, mojokerto, bosan, buah, melon...","[timesindonesia, mojokerto, bosan, buah, melon...",timesindonesia mojokerto bosan buah melon pasa...
3,Sejumlah Influencer Kepincut Pelayanan dan Fas...,"TIMESINDONESIA, BANYUWANGI – Luminor Hotel Ban...",07/09/2024 - 06:08,Wisata,TIMESINDONESIA BANYUWANGI Luminor Hotel Banyu...,timesindonesia banyuwangi luminor hotel banyu...,"[timesindonesia, banyuwangi, luminor, hotel, b...","[timesindonesia, banyuwangi, luminor, hotel, b...","[timesindonesia, banyuwangi, luminor, hotel, b...",timesindonesia banyuwangi luminor hotel banyuw...
4,5 Pantai Unik dan Cantik Masuk Kawasan TN Mata...,"TIMESINDONESIA, SUMBA TIMUR – Ternyata di Pula...",01/09/2024 - 14:16,Wisata,TIMESINDONESIA SUMBA TIMUR Ternyata di Pulau ...,timesindonesia sumba timur ternyata di pulau ...,"[timesindonesia, sumba, timur, ternyata, di, p...","[timesindonesia, sumba, timur, pulau, sumba, p...","[timesindonesia, sumba, timur, pulau, sumba, p...",timesindonesia sumba timur pulau sumba pantai ...


In [14]:
df_awal.to_csv("/content/drive/MyDrive/KULIAH/SEMESTER 7/PPW/berita_cleaned.csv",encoding='utf8', index=False)

## TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) dalah metode yang digunakan untuk menentukan seberapa penting sebuah kata dalam sebuah dokumen di dalam koleksi dokumen. Metode ini membantu dalam analisis teks dengan memberikan bobot pada kata-kata berdasarkan dua faktor utama: Frekuensi Kata (TF) dan Frekuensi Dokumen Terbalik (IDF).

Frekuensi Kata (TF) mengukur seberapa sering sebuah kata muncul dalam dokumen tertentu. Semakin sering kata tersebut muncul, semakin tinggi nilai TF-nya.

Frekuensi Dokumen Terbalik (IDF) mengukur seberapa penting sebuah kata di seluruh koleksi dokumen. Jika kata tersebut muncul di banyak dokumen, maka kata itu dianggap kurang penting untuk membedakan dokumen. IDF memberikan bobot lebih pada kata-kata yang jarang muncul di banyak dokumen, yang berarti kata-kata tersebut lebih spesifik untuk dokumen tertentu.

TF-IDF memberikan nilai akhir yang mencerminkan seberapa penting sebuah kata di dalam dokumen dalam konteks seluruh koleksi. Ini membantu mengidentifikasi kata-kata yang paling relevan dan informatif untuk analisis lebih lanjut, seperti dalam pencarian, klasifikasi, atau ekstraksi informasi.

In [17]:
data_tfidf = pd.read_csv("/content/drive/MyDrive/KULIAH/SEMESTER 7/PPW/berita_cleaned.csv")

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(data_tfidf['stemmed_text'])

In [19]:
# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)

,abad,ac,acara,ada,adat,adhi,adon,advertisement,adzan,agam,...,yoann,yogyakarta,yos,you,yougov,young,yuk,yusuf,zaman,zuhairi
0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.030472,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.23597
1,0.000000,0.069958,0.013973,0.034552,0.0,0.000000,0.0,0.000000,0.000000,0.015057,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000
2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.071053,0.0,0.0,0.0,0.035526,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000
5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.053174,0.0,0.0,0.0,0.053174,0.000000,0.000000,0.00000
6,0.000000,0.000000,0.079860,0.032912,0.0,0.138308,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000
7,0.041043,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.029874,0.041043,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.082087,0.082087,0.00000
8,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.030472,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.23597
9,0.000000,0.069958,0.013973,0.034552,0.0,0.000000,0.0,0.000000,0.000000,0.015057,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000
